# ipex official doc:
- Doc Link: https://intel.github.io/intel-extension-for-pytorch/cpu/2.0.100+cpu/
- Sample code:
```
import torch
import torchvision.models as models

model = models.resnet50(pretrained=True)
model.eval()
data = torch.rand(1, 3, 224, 224)

import intel_extension_for_pytorch as ipex
model = ipex.optimize(model)

with torch.no_grad():
  model(data)
```

# ipex.optimize()
- memory_format=torch.channels_last
- 算子融合：

| Origin    | After Fuse |
| -------- | ------- |
| nn.Conv1d, nn.BatchNorm1d | nn.Conv1d    |
| nn.Conv2d, nn.BatchNorm2d | nn.Conv2d    |
| nn.Conv3d, nn.BatchNorm3d | nn.Conv3d    |
| nn.Linear, nn.BatchNorm1d | nn.Linear    |
| nn.Linear, nn.BatchNorm2d | nn.Linear    |
| nn.Linear, nn.BatchNorm3d | nn.Linear    |


- 算子替换：

| Origin    | After replace |
| -------- | ------- |
| nn.Dropout | nn.Identity    |
| nn.LSTM | ipex._LSTM    |
| nn.Linear | ipex._IPEXLinear    |
| nn.Conv2d | ipex._IPEXConv2d    |
| nn.Conv3d | ipex._IPEXConv3d    |
| nn.Conv1d | ipex._IPEXConv1d    |
| nn.ConvTranspose2d | ipex._IPEXConvTranspose2d    |
| nn.ConvTranspose3d | ipex._IPEXConvTranspose3d    |

- 混合精度
...

# ipex.optimize()
## 函数调用
```
ipex.optimize(model)
    _convert_convNd_weight_memory_format()    // {ipex}, memory_format=torch.channels_last
    optimized_model = optimization.fuse(optimized_model, inplace=inplace) // {torch}: nn.Conv1d, nn.BatchNorm1d -> nn.Conv1d
    optimized_model = linear_bn_fuse(optimized_model, inplace=inplace) // {ipex}: nn.Linear, nn.BatchNorm1d -> nn.Linear
    utils._model_convert.replace_dropout_with_identity(optimized_model) // {ipex}: nn.Dropout -> nn.Identity
    optimized_model = utils._model_convert.convert_module_data_type(optimized_model, torch.bfloat16) // {ipex}: 混合精度
    optimized_model = utils._model_convert.convert_module_data_type(optimized_model, torch.half)   // {ipex}： 混合精度
    utils._model_convert.replace_lstm_with_ipex_lstm(optimized_model, optimized_optimizer)      // {ipex}: nn.LSTM -> nn._LSTM
    utils._weight_cast.weight_dtype_convert_with_ipex   // {ipex}: 算子替换
```


In [22]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
import intel_extension_for_pytorch as ipex
import torchinfo
import torch.fx.experimental.optimization as optimization

In [23]:
class MyModel_1(nn.Module):
    def __init__(self):
        super(MyModel_1, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(4, 6),nn.BatchNorm1d(6))
        self.layer2 = nn.Sequential(nn.Conv2d(6, 8, 2, 1), nn.BatchNorm2d(8))
        self.layer3 = nn.Dropout(0.1)
        self.layer4 = nn.LSTM(10, 20, 2)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        self.layer4(x, (torch.randn(2, 3, 20), torch.randn(2, 3, 20)))

m=MyModel_1()
m.eval()
m_ipex=ipex.optimize(m)

print("type(m)=",type(m))
print(torchinfo.summary(m,row_settings=["var_names"]))
print("\n")
print("type(m_ipex)=",type(m_ipex))
print(torchinfo.summary(m_ipex,row_settings=["var_names"]))

type(m)= <class '__main__.MyModel_1'>
Layer (type (var_name))                  Param #
MyModel_1 (MyModel_1)                    --
├─Sequential (layer1)                    --
│    └─Linear (0)                        30
│    └─BatchNorm1d (1)                   12
├─Sequential (layer2)                    --
│    └─Conv2d (0)                        200
│    └─BatchNorm2d (1)                   16
├─Dropout (layer3)                       --
├─LSTM (layer4)                          5,920
Total params: 6,178
Trainable params: 6,178
Non-trainable params: 0


type(m_ipex)= <class 'torch.fx.graph_module.GraphModule.__new__.<locals>.GraphModuleImpl'>
Layer (type (var_name))                  Param #
GraphModule (GraphModule)                --
├─Module (layer1)                        --
│    └─_IPEXLinear (0)                   1,982,695
├─Module (layer2)                        --
│    └─_IPEXConv2d (0)                   392
├─Identity (layer3)                      --
├─_LSTM (layer4)               

In [21]:
class MyModel_2(nn.Module):
    def __init__(self):
        super(MyModel_2, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(4, 6),nn.BatchNorm1d(6))
        self.layer2 = nn.Sequential(nn.Conv2d(6, 8, 2, 1), nn.BatchNorm2d(8))
        self.layer3 = nn.Dropout(0.1)
        self.layer4 = nn.LSTM(10, 20, 2)
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x[0] = 1 if x > 0 else 0             # add code to prevent torch.fx 
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        self.layer4(x, (torch.randn(2, 3, 20), torch.randn(2, 3, 20)))
        return x
m=MyModel_2()
m.eval()
m_ipex=ipex.optimize(m)

print("type(m)=",type(m))
print(torchinfo.summary(m,row_settings=["var_names"]))
print("\n")
print("type(m_ipex)=",type(m_ipex))
print(torchinfo.summary(m_ipex,row_settings=["var_names"]))

type(m)= <class '__main__.MyModel_2'>
Layer (type (var_name))                  Param #
MyModel_2 (MyModel_2)                    --
├─Sequential (layer1)                    --
│    └─Linear (0)                        30
│    └─BatchNorm1d (1)                   12
├─Sequential (layer2)                    --
│    └─Conv2d (0)                        200
│    └─BatchNorm2d (1)                   16
├─Dropout (layer3)                       --
├─LSTM (layer4)                          5,920
Total params: 6,178
Trainable params: 6,178
Non-trainable params: 0


type(m_ipex)= <class '__main__.MyModel_2'>
Layer (type (var_name))                  Param #
MyModel_2 (MyModel_2)                    --
├─Sequential (layer1)                    --
│    └─_IPEXLinear (0)                   1,982,695
│    └─BatchNorm1d (1)                   12
├─Sequential (layer2)                    --
│    └─_IPEXConv2d (0)                   392
│    └─BatchNorm2d (1)                   16
├─Identity (layer3)                

In [7]:
m=MyModel_2()
m.eval()
m_ipex=ipex.optimize(m)
print(torchinfo.summary(m))
print(torchinfo.summary(m_ipex))

Layer (type:depth-idx)                   Param #
MyModel_2                                --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       30
│    └─BatchNorm1d: 2-2                  12
├─Sequential: 1-2                        --
│    └─Conv2d: 2-3                       200
│    └─BatchNorm2d: 2-4                  16
├─Sequential: 1-3                        --
│    └─Linear: 2-5                       27
Total params: 285
Trainable params: 285
Non-trainable params: 0
Layer (type:depth-idx)                   Param #
MyModel_2                                --
├─Sequential: 1-1                        --
│    └─_IPEXLinear: 2-1                  1,982,695
│    └─BatchNorm1d: 2-2                  12
├─Sequential: 1-2                        --
│    └─_IPEXConv2d: 2-3                  392
│    └─BatchNorm2d: 2-4                  16
├─Sequential: 1-3                        --
│    └─_IPEXLinear: 2-5                  1,982,692
Total params: 3,965,807
Traina

/home/taosy/repo/intel/frameworks.ai.pytorch.ipex-cpu/build/Debug/packages/intel_extension_for_pytorch/frontend.py:474: UserWarning: Conv BatchNorm folding failed during the optimize process.
  warnings.warn("Conv BatchNorm folding failed during the optimize process.")
/home/taosy/repo/intel/frameworks.ai.pytorch.ipex-cpu/build/Debug/packages/intel_extension_for_pytorch/frontend.py:479: UserWarning: Linear BatchNorm folding failed during the optimize process.
  warnings.warn("Linear BatchNorm folding failed during the optimize process.")


In [5]:
class MyModel_1(nn.Module):
    def __init__(self):
        super(MyModel_1, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(50, 60, 3), nn.BatchNorm2d(60))
        self.layer2 = nn.Sequential(nn.Linear(60, 80),nn.BatchNorm1d(80))
        self.layer3 = nn.ReLU(80)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x
m=MyModel_1()
m.eval()

m_fx=torch.fx.symbolic_trace(m)
m_fuse=optimization.fuse(m, inplace=False)
m_ipex=ipex.optimize(m)
print("type(m)=",type(m))
print(torchinfo.summary(m,row_settings=["var_names"]))
print("\n")
print("type(m_fx)=",type(m_fx))
print(torchinfo.summary(m_fx,row_settings=["var_names"]))
print("\n")
print("type(m_fuse)=",type(m_fuse))
print(torchinfo.summary(m_fuse,row_settings=["var_names"]))
print("\n")
print("type(m_ipex)=",type(m_ipex))
print(torchinfo.summary(m_ipex,row_settings=["var_names"]))
m(torch.randn(1,50,62,62))

type(m)= <class '__main__.MyModel_1'>
Layer (type (var_name))                  Param #
MyModel_1 (MyModel_1)                    --
├─Sequential (layer1)                    --
│    └─Conv2d (0)                        27,060
│    └─BatchNorm2d (1)                   120
├─Sequential (layer2)                    --
│    └─Linear (0)                        4,880
│    └─BatchNorm1d (1)                   160
├─ReLU (layer3)                          --
Total params: 32,220
Trainable params: 32,220
Non-trainable params: 0


type(m_fx)= <class 'torch.fx.graph_module.GraphModule.__new__.<locals>.GraphModuleImpl'>
Layer (type (var_name))                  Param #
MyModel_1 (MyModel_1)                    --
├─Module (layer1)                        --
│    └─Conv2d (0)                        27,060
│    └─BatchNorm2d (1)                   120
├─Module (layer2)                        --
│    └─Linear (0)                        4,880
│    └─BatchNorm1d (1)                   160
├─ReLU (layer3)          

ValueError: expected 2D or 3D input (got 4D input)